In [21]:
import docker
import os
from rich import print

In [3]:
CONTAINER_AGENT_NAME = "intercode-agent"
CONTAINER_EVAL_NAME = "intercode-eval"
CONTAINER_IMAGE = "intercode"
CONTAINER_START_CMD = "bin/sh"

### Start Up a Container

In [4]:
client = docker.from_env()

In [5]:
if CONTAINER_AGENT_NAME in [container.name for container in client.containers.list()]:
    container_agent = client.containers.get(CONTAINER_AGENT_NAME)
else:
    container_agent = client.containers.run(CONTAINER_IMAGE, detach=True,
        name=CONTAINER_AGENT_NAME, tty=True)
container_agent.restart()

In [7]:
command = 'ls'
output = container_agent.exec_run(command).output.decode('utf-8')

In [8]:
print(output)

bin
dev
etc
home
lib
media
mnt
opt
proc
root
run
sbin
setup.sh
srv
sys
testbed
tmp
usr
var

### Run bash script via `exec_run`

In [24]:
os.path.dirname(os.path.abspath(__file__))

NameError: name '__file__' is not defined

In [20]:
";".join([x.replace("\n", "") for x in
          open("/Users/johnyang/Desktop/Research/intercode/bash/setup.sh", "r").readlines()
          if not x.startswith("#") and len(x) > 0 and x != "\n"])

'rm -rf /testbed;mkdir -p /testbed/dir1/subdir1/subsubdir1;mkdir -p /testbed/dir1/subdir2;mkdir -p /testbed/dir2/subdir1;mkdir -p /testbed/dir2/subdir2/subsubdir1;mkdir -p /testbed/dir3/subdir1/subsubdir1;mkdir -p /testbed/dir3/subdir2;echo "Hello, World!" > /testbed/dir1/textfile1.txt;echo "Another text file" > /testbed/dir2/subdir1/textfile2.txt;echo "Yet another text file" > /testbed/dir3/subdir1/subsubdir1/textfile3.txt;echo "Text file 4" > /testbed/dir1/subdir1/textfile4.txt;echo "Text file 5" > /testbed/dir2/subdir2/textfile5.txt;echo "Text file 6" > /testbed/dir3/textfile6.txt;echo -e "#!/bin/sh\\necho \'Shell script 1\'" > /testbed/dir1/subdir1/shellscript1.sh;echo -e "#!/bin/sh\\necho \'Shell script 2\'" > /testbed/dir2/shellscript2.sh;echo -e "#!/bin/sh\\necho \'Shell script 3\'" > /testbed/dir3/subdir1/subsubdir1/shellscript3.sh;echo -e "#!/bin/sh\\necho \'Shell script 4\'" > /testbed/dir1/subdir1/subsubdir1/shellscript4.sh;echo -e "#!/bin/sh\\necho \'Shell script 5\'" > /te

In [15]:
f = open("/Users/johnyang/Desktop/Research/intercode/bash/setup.sh", "r")
for line in f.readlines():
    print("hi", line)

hi #!/bin/sh

hi rm -rf /testbed

hi

hi mkdir -p /testbed/dir1/subdir1/subsubdir1

hi mkdir -p /testbed/dir1/subdir2

...

hi chmod +x /testbed/dir3/subdir1/pythonscript3.py

hi chmod +x /testbed/dir1/subdir1/pythonscript4.py

hi

hi echo -e '{\n  "key1": "value1",\n  "key2": "value2",\n  "key3": "value3"\n}' > 
/testbed/dir1/subdir1/jsonfile1.json

hi echo -e 'column1,column2,column3\nvalue1,value2,value3\nvalue4,value5,value6' > 
/testbed/dir3/subdir2/csvfile1.csv

In [10]:
!ls

docker.ipynb     playground.ipynb


### Playing with Sockets

In [65]:
_, socket_agent = container_agent.exec_run(
    cmd=CONTAINER_START_CMD, stdin=True, tty=True, stream=True, socket=True)

In [68]:
actions = ['pwd', 'touch hello.txt', 'ls', 'mkdir folder1', 'mkdir -p folder3/folder4/file2.txt' 'mkdir folder2', 'rm -rf folder2']
actions = ['sleep 2; echo \"hi\"']
for action in actions:
    socket_agent._sock.send(action.encode('utf-8') + b"\n")
    time.sleep(0.5)
    output = socket_agent._sock.recv(100000).decode("utf-8")
    print(f"{output}\n-----")

sleep 2; echo "hi"

-----


### Playing with container `diff`

In [67]:
changes = container_agent.diff()
if changes and len(changes) > 0:
    for change in changes:
        print(change['Path'], change['Kind'])
else:
    print("No changes")

/folder1 1
/folder3 1
/folder3/folder4 1
/folder3/folder4/file2.txtmkdir 1
/hello.txt 1


In [87]:
changes = [f"{x['Path']} - {x['Kind']}" for x in container_agent.diff()]
changes

['/hello.txt - 1',
 '/folder3 - 1',
 '/folder3/folder4 - 1',
 '/folder3/folder4/file2.txtmkdir - 1',
 '/folder1 - 1']

In [91]:
copy = container_agent.diff()[:3]
changes_2 = [f"{x['Path']} - {x['Kind']}" for x in copy]
changes_2

['/hello.txt - 1', '/folder1 - 1', '/folder3 - 1']

In [93]:
set(changes_2) - set(changes)

set()

In [94]:
set(changes) - set(changes_2)

{'/folder3/folder4 - 1', '/folder3/folder4/file2.txtmkdir - 1'}